In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# loading data
preds = np.load("../results/preds.np")
label = np.load("../results/label.np")
print(preds.shape)

# create prd dataframe
preds = pd.Series(preds.tolist())
res = pd.DataFrame({'pred':preds, 'label': label})
#res["count"] = res.apply(lambda row: len(res[res["label"] == row["label"]]), axis=1)
#res = res[res["count"] > 90]


print(len(res))

(124, 128)
124


In [4]:
def create_positif(df, nbr):
    duets = []
    list_label = list(set(df["label"].tolist()))
    
    for elem in range(nbr):
        chosen_label = random.choice(list_label)
        
        # select duet
        select = df[df["label"] == chosen_label]
        samp = select.sample(2)
        samp = samp["pred"].tolist()
        d = (samp[0], samp[1])
        
        duets.append(d)
        
    return duets
        
        

def create_negatif(df, nbr):
    duets = []
    list_label = list(set(df["label"].tolist()))
    
    for elem in range(nbr):
        chosen_label = random.choice(list_label)
        
        # select duet
        pos = df[df["label"] == chosen_label]
        neg = df[~(df["label"] == chosen_label)]
        
        p_samp = pos.sample(1)
        n_samp = neg.sample(1)
        
        p_samp = p_samp["pred"].tolist()[0]
        n_samp = n_samp["pred"].tolist()[0]
        
        
        d = (p_samp, n_samp)
        
        duets.append(d)
        
    return duets
        



In [8]:
positives = create_positif(res, 100)
negatives = create_negatif(res, 100)

print(len(positives))
print(len(negatives))

100
100


In [22]:
# Evaluation of the model

limit_dist = 0.0000158

tp = 0
tn = 0
fp = 0
fn = 0

# Evaluate positif
cosine_pos = []
eucli = []
for pos in positives:
    t1 = np.array([pos[0]])
    t2 = np.array([pos[1]])
    cosine_pos.append(cosine_similarity(t1, t2)[0][0])
    eucli.append(np.linalg.norm(t1[0]-t2[0]))
    if eucli[-1] <= limit_dist:
        tp += 1
    else:
        fn += 1

c = np.array(cosine_pos)
e = np.array(eucli)
print("Moyenne consine {} for {}".format(c.mean(), len(c)))
print("Moyenne eucli {} for {}".format(e.mean(), len(e)))


cosine_neg = []
eucli = []
for neg in negatives:
    t1 = np.array([neg[0]])
    t2 = np.array([neg[1]])
    cosine_neg.append(cosine_similarity(t1, t2)[0][0])
    eucli.append(np.linalg.norm(t1[0]-t2[0]))
    
    if eucli[-1] > limit_dist:
        tn += 1
    else:
        fp += 1
    
c = np.array(cosine_neg)
e = np.array(eucli)
print("Moyenne consine {}".format(c.mean()))
print("Moyenne eucli {} for {}".format(e.mean(), len(e)))

print("\n\n")

accuracy = (tp + tn) / 10000
precision = tp /(fp + tp)
recall = tp / (fn+tp)
f1 = 2*((precision*recall)/(precision+recall))

print("accuracy: {}".format(accuracy))
print("precision: {}".format(precision))
print("recall: {}".format(recall))
print("f1: {}".format(f1))

Moyenne consine 0.8371299678342687 for 100
Moyenne eucli 1.9398523623765156e-05 for 100
Moyenne consine 0.8743505164951454
Moyenne eucli 1.5819896028691957e-05 for 100



accuracy: 0.0098
precision: 0.49382716049382713
recall: 0.8
f1: 0.6106870229007633


In [16]:
t1 = res["pred"].tolist()[0]
t2 = res["pred"].tolist()[1000]

t2[0] += 3

print((t1 == t2))

False
